1. **Estimated Depth of a Decision Tree**:
In a binary tree, the maximum depth \( d \) (unrestricted) can be estimated as \( \log_2(n) \), where \( n \) is the number of instances. So for 1 million instances, the estimated depth would be \( \log_2(1,000,000) \) which is roughly 20. However, in practice, it might be less because of early stopping conditions or the fact that not every instance will require a leaf.

2. **Gini Impurity**:
Typically, the Gini impurity of a child node is lower than its parent because the purpose of the split is to clarify the classes. If a split is made, it's because it's believed that this split can better separate the instances. However, it's not guaranteed to always be lower. It's just usually the case.

3. **Reducing max_depth for Overfitting**:
Yes, if a Decision Tree is overfitting the training set, reducing the maximum depth (`max_depth`) can help. This makes the tree more general and prevents it from fitting to the noise in the training data.

4. **Scaling Input Features for Decision Trees**:
Decision Trees are not sensitive to the scale of input features. Therefore, scaling the input features will not have any effect on its performance, whether it's underfitting or overfitting.

5. **Training Time Estimate**:
The training time complexity of a Decision Tree is \( O(n \times m \log(m)) \), where \( n \) is the number of features and \( m \) is the number of instances. If you multiply \( m \) by 10, the training time will be multiplied roughly by \( 10 \log(10) \) which is about 23. Therefore, if it takes 1 hour for 1 million instances, it might take roughly 23 hours for 10 million instances.

6. **Presort for Training Speed**:
Setting `presort=True` can speed up training on smaller datasets or for smaller depths, but for larger datasets like 100,000 instances, it can considerably slow down training.

7. **Training and Fine-tuning on Moons Dataset**:
I won't be able to execute the code here directly, but I can guide you step by step:

In [ ]:
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# a
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

# b
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# c
params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)

# d
y_pred = grid_search_cv.best_estimator_.predict(X_test)
print(accuracy_score(y_test, y_pred))

8. **Growing a Forest**:

In [ ]:
from sklearn.model_selection import ShuffleSplit
from scipy.stats import mode

# a
n_trees = 1000
n_instances = 100
mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

# b
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

print(np.mean(accuracy_scores))

# c
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

# d
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))